# Aprendendo a utilizar o ADK (Agent Development Toolkit) do watsonx Orchestrate

O objetivo desse notebook é demonstrar como utilizar as principais funcionalidades do ADK (Agent Development Toolkit) do watsonx Orchestrate. Nas próximas células vamos configurar o ADK e criar os principais componentes de um agente na plataforma, uma Knowledge Base, uma Tool e o Agent em si.  

**Observação 1: Esse notebook foi planejado para ser executado dentro de um projeto do watsonx.ai Studio, entretanto ele é facilmente adaptável para rodar localmente.**

**Observação 2: No repositório do GitHub desse notebook, foi disponibilizado para você um PDF exemplo de bula para ser usado como Knowledge Base. Adicione-o também ao seu projeto ou use algum outro PDF de sua preferência.**

### Instalando o pacote `ibm-watsonx-orchestrate` 

Vamos começar instalando o ADK do watsonx Orchestrate. Com ele vamos ter acesso aos comandos necessários para desenvolver os agentes e todos os outros componentes que eles utilizam. 

In [ ]:
! pip install ibm-watsonx-orchestrate | tail -1


### Instalação do pyyaml

O pacote `pyyaml` é uma implementação em Python para o formato de serialização YAML. O ADK utiliza arquivos YAML para definir as configurações dos agentes.


In [ ]:
! pip install pyyaml | tail -1


## Execução do comando `orchestrate`

O comando `!orchestrate --help` é usado para visualizar a ajuda do comando `orchestrate`. 
O uso desse comando é essencial para entender como utilizar a ferramenta `orchestrate` em linha de comando.


In [ ]:
! orchestrate --help


### Adicionar um novo ambiente do Watsonx Orchestrate

O comando `!orchestrate env add` é usado para adicionar um novo ambiente do Watsonx Orchestrate ao seu ADK. Esse ambiente é uma instância do serviço que permite a execução dos agentes.

In [ ]:
! orchestrate env add -n techzone-037 -u <url> # Format: https://api.us-south.watson-orchestrate.cloud.ibm.com/instances/<instanceid>

[INFO] - Creating config file at location "/home/wsuser/.config/orchestrate/config.yaml"
[DEBUG] - Setting default config data
[INFO] - Environment 'techzone-037' has been created



### Ativação do Ambiente do Watsonx Orchestrate

O comando `!orchestrate env activate` é usado para ativar um ambiente específico do Watsonx Orchestrate. 
Este ambiente deve ter sido previamente adicionado utilizando o comando `!orchestrate env add`. 
Para ativar o ambiente, é necessário fornecer o nome do ambiente (`techzone-037`) e uma chave de API (`<apikey>`) válida para autenticação.


In [ ]:
! orchestrate env activate techzone-037 --api-key <apikey>

[INFO] - Creating config file at location "/home/wsuser/.cache/orchestrate/credentials.yaml"
[DEBUG] - Setting default credentials data
[INFO] - Environment 'techzone-037' is now active


## Criação de uma nova Knowledge Base

A *Knowledge Base* é um componente do ADK que armazena os documentos e informações relevantes para o funcionamento de um agente. Nas próximas células vamos passar por todos os passos para a criação de uma *Knowledge Base*, começando pela obtenção dos documentos. Esse notebook foi executado em um ambiente da Cloud (watsonx.ai Studio) então vamos pegar os documentos de um bucket do IBM Cloud Object Storage. Se você estiver executando esse notebook localmente, você pode pular esses passos.

### Acesso ao Arquivo no IBM Cloud Object Storage

Nesta célula, estamos acessando um arquivo armazenado no IBM Cloud Object Storage. O arquivo em questão é uma bula de remédio, denominado `bula_buscoduo.pdf`, que está armazenado em um bucket específico. Esse arquivo foi obtido no [bulário do Ministério da Saúde](https://consultas.anvisa.gov.br/#/bulario/q/?nomeProduto=BUSCODUO).

#### Configuração do Cliente IBM Cloud Object Storage

Para acessar o arquivo, primeiro configuramos o cliente IBM Cloud Object Storage utilizando a biblioteca `ibm_boto3`. Essa configuração inclui a definição da chave de API (`ibm_api_key_id`), o endpoint de autenticação (`ibm_auth_endpoint`), a versão de assinatura (`signature_version`) e o endpoint da URL (`endpoint_url`).

#### Definição do Bucket e Objeto

Em seguida, definimos o nome do bucket (`bucket`) e a chave do objeto (`object_key`) que desejamos acessar. Neste caso, o bucket é denominado `{{BUCKET_NAME}}` e o objeto é o arquivo `bula_buscoduo.pdf`.

#### Salvar o Arquivo

O arquivo é salvo com o mesmo nome (`bula_buscoduo.pdf`) no diretório de trabalho. Isso permite que o arquivo seja acessado e utilizado posteriormente nas células do Jupyter Notebook.

Essa etapa é crucial para garantir que o arquivo esteja disponível para uso nos agentes do Watsonx Orchestrate, que serão configurados nas células subsequentes.

In [ ]:

import os
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): 
    return 0

# Configuração do cliente IBM Cloud Object Storage
cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='{{API_KEY}}',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='{{ENDPOINT_URL}}'
)

# Definição do bucket e objeto a ser acessado
bucket = '{{BUCKET_NAME}}'
object_key = 'bula_buscoduo.pdf'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Salvar o arquivo no diretório de trabalho
with open('bula_buscoduo.pdf', 'wb') as file:
    file.write(streaming_body_1.read())


### Listagem de Arquivos do Diretório Atual

O comando `! ls` é usado para listar os arquivos e diretórios presentes no diretório atual do *Jupyter Notebook*. 
Este comando é útil para verificar o conteúdo do diretório de trabalho e garantir que o arquivo foi corretamente salvo.


In [8]:
! ls

bula_buscoduo.pdf



### Criação de arquivo YAML para *Knowledge Base*

Nesta célula, criamos um arquivo YAML que representa a configuração de uma *knowledge base* para uso no Watsonx Orchestrate. Neste caso, a *knowledge base* denominada `kb_buscoduo` contém uma referência a um documento PDF, especificamente a bula do medicamento Buscoduo.

O código em Python utiliza a biblioteca `yaml` para serializar o dicionário `data` em um formato YAML e salva em um arquivo denominado `kb_buscoduo.yaml`.Esse arquivo YAML será usado para importar a *knowledge base* no ambiente do Watsonx Orchestrate.


In [28]:
import yaml

data = {
    'spec_version': 'v1',
    'kind': 'knowledge_base',
    'name': 'kb_buscoduo',
    'description': 'Base de conhecimento com a bula do medicamento Buscoduo.',
    'documents': [
        '/home/wsuser/work/bula_buscoduo.pdf'
    ]
}

with open('kb_buscoduo.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)


### Importação de *Knowledge Base* no Watsonx Orchestrate

Nesta célula, importamos a *knowledge base* criada anteriormente no Watsonx Orchestrate. 
O comando `! orchestrate knowledge-bases import -f kb_buscoduo.yaml` é usado para importar a *knowledge base* a partir do arquivo YAML criado na célula anterior.
Ao executar este comando, a *knowledge base* é registrada no ambiente do Watsonx Orchestrate, tornando-a disponível para uso no agente.


In [29]:
! orchestrate knowledge-bases import -f kb_buscoduo.yaml

[INFO] - Successfully imported knowledge base 'kb_buscoduo'



 ### Listagem de Knowledge Bases
 
 O comando `!orchestrate knowledge-bases list` é usado para listar todas as *knowledge bases* atualmente configuradas no ambiente do Watsonx Orchestrate, para uso nos agentes
 

In [ ]:
! orchestrate knowledge-bases list


### Remoção de *Knowledge Base* no Watsonx Orchestrate
A execução desta célula irá remover permanentemente a *knowledge base* `kb_buscoduo`. 

#### ATENÇÃO: Esta célula está comentada e só deve ser executada se necessário.


In [ ]:
##! orchestrate knowledge-bases remove --name kb_buscoduo

[INFO] - Successfully removed knowledge base 'kb_buscoduo'



## Exemplo de Ferramenta (Tool): Pedido de Compra de Medicamento

### Descrição
A ferramenta `place_order_tool` é projetada para realizar pedidos de compra de medicamentos. Ela recebe dois parâmetros: `product_name` (nome do produto) e `address` (endereço de entrega). A ferramenta verifica se ambos os parâmetros são fornecidos e, se sim, retorna uma mensagem de confirmação do pedido. Caso contrário, ela retorna uma mensagem de falha no pedido.

### Parâmetros
- **product_name** (str): Nome do medicamento a ser comprado.
- **address** (str): Endereço para entrega do medicamento.

### Retorno
- Uma string indicando o resultado do pedido: confirmação de pedido realizado ou falha no pedido.


In [15]:
def place_order_tool(product_name:str, address:str) -> str:
    if product_name != "" and address != "":
        return "Pedido realizado de {} para o endereço: {}".format(product_name, address)
    else:
        return "O pedido falhou"

In [16]:
place_order_tool("Buscoduo", "Rua Tutoia 1157")

'Pedido realizado de Buscoduo para o endereço: Rua Tutoia 1157'


### Criação do arquivo Python para a ferramenta `place_order_tool` 

A célula a seguir define a ferramenta `place_order_tool` utilizando a biblioteca `ibm_watsonx_orchestrate`. A implementação da ferramenta é feita através de uma função Python decorada com `@tool`, que é uma função decoradora fornecida pela biblioteca `ibm_watsonx_orchestrate` para definir ferramentas. A função `place_order_tool` é então escrita em um arquivo Python chamado `place_order_tool.py`.


In [31]:
function = '''
from ibm_watsonx_orchestrate.agent_builder.tools import tool, ToolPermission

@tool(name="place_order_tool", description="Ferramenta para criar um pedido de compra de um medicamento.", permission=ToolPermission.ADMIN)
def place_order_tool(product_name:str, address:str) -> str:
    if product_name != "" and address != "":
        return "Pedido realizado de {} para o endereço: {}".format(product_name, address)
    else:
        return "O pedido falhou"
'''

with open("place_order_tool.py", "w", encoding="utf-8") as f:
    f.write(function)

### Importação da Ferramenta `place_order_tool`

A célula a seguir importa a ferramenta `place_order_tool` utilizando a biblioteca `ibm_watsonx_orchestrate`. Essa ferramenta foi definida anteriormente no arquivo `place_order_tool.py` e é projetada para criar um pedido de compra de um medicamento com base nos parâmetros fornecidos: `product_name` e `address`.

In [32]:
! orchestrate tools import -f place_order_tool.py -k python

[INFO] - Existing Tool 'place_order_tool' found. Updating...
[INFO] - Tool 'place_order_tool' updated successfully



## Criação do Arquivo de Configuração do Agente

Nesta célula, criamos um arquivo de configuração para o agente `agente_bulas` utilizando a biblioteca `yaml`. O arquivo de configuração contém informações sobre o agente, incluindo sua versão, tipo, nome, modelo de linguagem, estilo, descrição, instruções, colaboradores, ferramentas e base de conhecimento.

### Parâmetros de Configuração
- **spec_version**: Versão da especificação do agente.
- **kind**: Tipo do agente (native ou external).
- **name**: Nome do agente.
- **llm**: Modelo de linguagem utilizado pelo agente.
- **style**: Estilo do agente (react ou default).
- **description**: Descrição do agente. Esse campo vai ser usado por outros agentes para entender qual o papel deste colaborador.
- **instructions**: Instruções para o agente. Esse campo vai ser usado pelo próprio agente para entender o que deve ser feito.
- **collaborators**: Lista de colaboradores do agente.
- **tools**: Lista de ferramentas disponíveis para o agente.
- **knowledge_base**: Lista de bases de conhecimento do agente.


In [22]:
import yaml

data = {
    'spec_version': 'v1',
    'kind': 'native',
    'name': 'agente_bulas',
    'llm': 'watsonx/meta-llama/llama-3-3-70b-instruct',
    'style': 'react',
    'description': (
        "Agente para tirar dúvidas sobre o medicamento Buscoduo."
    ),
    'instructions': (
        "Você é um agente amigável que responde dúvidas do usuário sobre o Buscoduo, sempre consultando a bula.\n"
        "Se o usuário perguntar sobre um outro assunto, diga que você não responde sobre outros temas.\n"
        "Responda sempre em Português do Brasil."
    ),
    'collaborators': [
    ],
    'tools': [
        "place_order_tool"
    ],
    'knowledge_base': [
        "kb_buscoduo"
    ],
}

# Write to YAML
with open('agent_buscoduo.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

### Importação do Agente

Nesta célula, importamos o agente definido no arquivo `agent_buscoduo.yaml`. Esse arquivo YAML contém a configuração do agente, incluindo suas ferramentas, base de conhecimento e instruções.

In [30]:
! orchestrate agents import -f agent_buscoduo.yaml

[INFO] - Existing Agent 'agente_bulas' found. Updating...
[INFO] - Agent 'agente_bulas' updated successfully


### Listagem de Agentes
A célula a seguir lista todos os agentes disponíveis. Isso é feito para garantir que o agente `agente_bulas` tenha sido importado corretamente e esteja pronto para uso. Anote o id do agente criado pois no passo seguinte vamos enviar uma mensagem de teste.



In [0]:
! orchestrate agents list -v

## Testando o agente pela API

Nos passos seguintes vamos enviar uma mensagem ao agente criado para testar as respostas.


### Autenticação com o watsonx Orchestrate
Nesta célula, vamos criar um token de autenticação do IAM para utilizar na API do watsonx Orchestrate.


In [0]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

apikey = '<apikey>'

authenticator = IAMAuthenticator(apikey)
token = authenticator.token_manager.get_token()


### Envio de Requisição para o watsonx Orchestrate
Nesta célula, enviamos uma requisição POST para a API do watsonx Orchestrate, utilizando a biblioteca `requests`. A requisição é enviada para o endpoint de completions do agente, passando como parâmetro o ID da instância e o ID do agente.
Os headers da requisição incluem o token de autenticação obtido anteriormente e a chave de API. O corpo da requisição é um JSON que contém a mensagem do usuário, que é enviada para o agente para processamento.


In [0]:
import requests

response = requests.post("https://api.us-south.watson-orchestrate.cloud.ibm.com/instances/<instance-id>/v1/orchestrate/<agent-id>/chat/completions",
              headers={"Authentication": "Bearer "+token, "IAM-API_KEY": apikey, "Content-Type": "application/json", "Accept": "application/json"},
             data='{"stream": true, "messages": [{"role": "user", "content": "Buscoduo funciona para colicas menstruais?"}]}')
response.text

### Observação: Modo Streaming

Na célula anterior ativamos a propriedade `stream`, ela permite que sejam vistos todos os eventos de processamento do agente, como a chamada para as tools.